<a href="https://colab.research.google.com/github/facundocarballo/PogramacionConcurrente/blob/main/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C++

El programa realiza un producto matricial de dos formas distintas, de forma secuencial y concurrente usando hilos. Comparando el resultado obtenido y el tiempo de respuesta de cada implementacion.

Para ejecutarlo hay que pasarle como parametro el orden de las matrices que vamos a multiplicar.

In [1]:
%%writefile matriz.cpp
#include <chrono>
#include <iostream>
#include <string>
#include <thread>
#include <tuple>
#include <vector>

#define CANTIDAD_NUMEROS 65
#define LIMITE 32

#define ORDEN_MAXIMO 20
#define ORDEN_MINIMO 5

#define MATRIZ_A "A"
#define MATRIZ_B "B"
#define MATRIZ_SECUENCIAL "CS"
#define MATRIZ_HILOS "CH"

void Ayuda() 
{
  printf(
      "Enviar un parametro indicando la cantidad de filas y columnas de la "
      "matriz.\n");
  printf("------\n");
  printf("Ejemplo:\n");
  printf("./matrix 3\n");
  printf("------\n");
  printf("Esto hara que el programa trabaje con una matrix de 3x3\n");
  printf("------\n");
  printf("Los ordenes de matriz aceptados van desde %d hasta %d.\n", ORDEN_MINIMO, ORDEN_MAXIMO);
}

class Matriz 
{
 private:
  int orden_;
  int** matriz_;
  std::string nombre_;

  void ImprimirConEstilo(int n) 
  {
    if (n > 99) 
    {
      std::cout << " " << n << "  ";
    } else if (n > 9) 
    {
      std::cout << "  " << n << "  ";
    } else if (n > 0) 
    {
        std::cout << "   " << n << "  ";
    }
    else if (n > -10) 
    {
      std::cout << "  " << n << "  ";
    } else if (n > -100) 
    {
      std::cout << " " << n << "  ";
    } else 
    {
        std::cout << n << "  ";
    }
  }
  void MultiplicarFila(int fila, Matriz* B, Matriz* C) 
  {
    for (int i = 0; i < this->orden_; i++) 
    {
      C->matriz_[fila][i] = this->matriz_[fila][i] * B->matriz_[fila][i];
    }
  }

 public:
  Matriz(int orden, std::string nombre) 
  {
    this->orden_ = orden;
    this->nombre_ = nombre;
    matriz_ = new int*[orden];
    for (int i = 0; i < orden; i++) 
    {
      matriz_[i] = new int[orden];
    }
  }
  void Random() 
  {
    for (int i = 0; i < this->orden_; i++) 
    {
      for (int j = 0; j < this->orden_; j++) 
      {
        matriz_[i][j] = rand() % CANTIDAD_NUMEROS + (-LIMITE);
      }
    }
  }
  void Imprimir() 
  {
    std::cout << "Matriz: " << this->nombre_ << std::endl;
    for (int i = 0; i < this->orden_; i++) 
    {
      for (int j = 0; j < this->orden_; j++) 
      {
        this->ImprimirConEstilo(this->matriz_[i][j]);
      }

      std::cout << std::endl;
    }
  }
  std::tuple<Matriz*, std::chrono::_V2::system_clock::duration>
  Multiplicar(Matriz* B) 
  {
    Matriz* C = new Matriz(this->orden_, MATRIZ_SECUENCIAL);
    auto comienzo = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < this->orden_; i++) 
    {
      for (int j = 0; j < this->orden_; j++) 
      {
        C->matriz_[i][j] = this->matriz_[i][j] * B->matriz_[i][j];
      }
    }
    auto fin = std::chrono::high_resolution_clock::now();
    return std::make_tuple(C, fin - comienzo);
  }
  std::tuple<Matriz*, std::chrono::_V2::system_clock::duration>
  MultiplicarConcurrente(Matriz* B) 
  {
    Matriz* C = new Matriz(this->orden_, MATRIZ_HILOS);
    std::vector<std::thread> hilos;
    auto comienzo = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < this->orden_; i++) 
    {
      hilos.emplace_back(&Matriz::MultiplicarFila, this, i, B, C);
    }

    for (auto& h : hilos) 
    {
      h.join();
    }

    auto fin = std::chrono::high_resolution_clock::now();

    return std::make_tuple(C, fin - comienzo);
  }
  bool EsIgualA(Matriz* B)
  {
    bool res = true;
    int i = 0;
    int j = 0;
    while(res && i < this->orden_)
    {
      while(res && j < this->orden_)
      {
        if (this->matriz_[i][j] != B->matriz_[i][j])
        {
          res = false;
        }
        j++;
      }
      i++;
    }
    return res;
  }
};

void ImprimirTiempo(std::chrono::milliseconds tiempo) 
{
  std::cout << "Hecho en: " << tiempo.count() << " ms" << std::endl;
}

Matriz* MultiplicacionSecuencial(Matriz* A, Matriz* B) 
{
  std::cout << "\nMultiplicacion Secuencial" << std::endl;
  Matriz* C;
  std::chrono::_V2::system_clock::duration tiempo;
  std::tie(C, tiempo) = A->Multiplicar(B);
  C->Imprimir();
  ImprimirTiempo(std::chrono::duration_cast<std::chrono::milliseconds>(tiempo));

  return C;
}

Matriz* MultiplicacionConcurrente(Matriz* A, Matriz* B) 
{
  std::cout << "\nMultiplicacion Concurrente" << std::endl;
  Matriz* D;
  std::chrono::_V2::system_clock::duration tiempo2;
  std::tie(D, tiempo2) = A->MultiplicarConcurrente(B);
  D->Imprimir();
  ImprimirTiempo(
      std::chrono::duration_cast<std::chrono::milliseconds>(tiempo2));

  return D;
}

int main(int argc, char* argv[]) 
{
  if (argc != 2) 
  {
    Ayuda();
    return EXIT_FAILURE;
  }

  int n = std::stoi(argv[1]);

  if (n < ORDEN_MINIMO || n > ORDEN_MAXIMO)
  {
    Ayuda();
    return EXIT_FAILURE;
  }

  Matriz* A = new Matriz(n, MATRIZ_A);
  A->Random();
  A->Imprimir();

  Matriz* B = new Matriz(n, MATRIZ_B);
  B->Random();
  B->Imprimir();

  Matriz* C = MultiplicacionSecuencial(A, B);
  Matriz* D = MultiplicacionConcurrente(A, B);

  if (C->EsIgualA(D))
  {
    std::cout << "Las matrices son iguales." << std::endl;
  }
  else
  {
    std::cout << "Las matrices son diferentes." << std::endl;
  }

  return EXIT_SUCCESS;
}

Writing matriz.cpp


Compilamos el codigo C++ y generamos un binario.

In [2]:
!g++ matriz.cpp -lpthread -o matriz.bin

Corremos el programa en segundo plano y enviamos toda la salida del programa al archivo "salidaC"



In [3]:
!nohup ./matriz.bin 5 1> salidaC 2> /dev/null & 

Visualizamos lo que hay dentro del archivo "salidaC"

In [4]:
!cat salidaC

Matriz: A
 -19    29     5    -2    21  
   8    19    10   -18    14  
 -30    10   -32   -28     6  
 -31     3     4    30   -21  
  14   -24    10    -3    10  
Matriz: B
 -27   -30   -29    30    -2  
 -23   -20    -5    16    12  
 -15    -9    -1    29     5  
 -18    31   -16   -18   -28  
  25   -14     7    -2   -17  

Multiplicacion Secuencial
Matriz: CS
 513  -870  -145   -60   -42  
-184  -380   -50  -288   168  
 450   -90    32  -812    30  
 558    93   -64  -540   588  
 350   336    70     6  -170  
Hecho en: 0 ms

Multiplicacion Concurrente
Matriz: CH
 513  -870  -145   -60   -42  
-184  -380   -50  -288   168  
 450   -90    32  -812    30  
 558    93   -64  -540   588  
 350   336    70     6  -170  
Hecho en: 0 ms
Las matrices son iguales.


# Java

La clase MatrixMultiplier brinda 2 metodos principales que nos permiten calcular la multiplicacion entre 2 matrices de formas diferentes; secuencial o utilizando hilos.

In [5]:
%%writefile MatrixMultiplier.java

import java.time.Duration;
import java.time.Instant;
import java.util.Random;

public class MatrixMultiplier implements Runnable 
{

  private int[][] matrix1;
  private int[][] matrix2;
  private int[][] result;
  private int row;

  public MatrixMultiplier(int[][] matrix1, int[][] matrix2, int[][] result, int row) 
  {
    this.matrix1 = matrix1;
    this.matrix2 = matrix2;
    this.result = result;
    this.row = row;
  }
  
  @Override
  public void run() 
  {
    multiply(row);
  }
  
  
  public static int[][] multiplyNormal(int[][] matrix1, int[][] matrix2) 
  {
    if(!canMultiply(matrix1,matrix2)) 
    {
    	throw new RuntimeException();
    }
    
    int n = matrix1.length;
    int[][] result = new int[n][n];
    for (int i = 0; i < n; i++) 
    {
      for (int j = 0; j < n; j++) 
      {
        for (int k = 0; k < n; k++) 
        {
          result[i][j] += matrix1[i][k] * matrix2[k][j];
        }
      }
    }
    return result;
  }
  
  /*
   * Esta es la funcion que se encarga de solo multiplicar una fila 
   * para que asi cada hilo cumpla con su respectiva multiplicacion
   */
  public void multiply(int row) 
  {
    int n = matrix1.length;
    for (int j = 0; j < n; j++) 
    {
      for (int k = 0; k < n; k++) 
      {
        result[row][j] += matrix1[row][k] * matrix2[k][j];
      }
    }
  }
  
  /*
   * Esta funcion implementa un Dessing Pattern de Linea de Ensamblaje donde
   * se realiza la multiplicacion de 2 matrices donde por cada fila hay 1 hilo
   * que se encarga de su multiplicacion 
   */
  public static int[][] multiplyThreads(int[][] matrix1, int[][] matrix2) 
  {
	
	  if(!canMultiply(matrix1,matrix2))
    {
      throw new RuntimeException();
    }
		
	
	  int n = matrix1.length;
	
	  MatrixMultiplier[] multipliers = new MatrixMultiplier[n];
    Thread[] threads = new Thread[n];
    int[][] CH = new int[n][n];
    for (int i = 0; i < n; i++) 
    {
      multipliers[i] = new MatrixMultiplier(matrix1, matrix2, CH, i);
      threads[i] = new Thread(multipliers[i]);
      threads[i].start();
    }
    try 
    {
      for (int i = 0; i < n; i++) 
      {
        threads[i].join();
      }
    } 
    catch (InterruptedException e) 
    {
      e.printStackTrace();
    }
    
    return CH;
    
  }

  public static int[][] generateRandomMatrix(int n) 
  {
    int[][] matrix = new int[n][n];
    Random rand = new Random();
    
    for (int i = 0; i < n; i++) 
    {
      for (int j = 0; j < n; j++) 
      {
        matrix[i][j] = rand.nextInt(65) - 32;
      }
    }
    return matrix;
  }

  public static void printMatrix(int[][] matrix) 
  {
	  int n = matrix.length;
	  int maxLength = Integer.MIN_VALUE;
	  
	  for (int i = 0; i < n; i++) 
	  {
	    for (int j = 0; j < n; j++) 
	    {
	      int length = String.valueOf(matrix[i][j]).length();
	      if (length > maxLength) 
	      {
	        maxLength = length;
	      }
	    }
	  }
	  
	  String format = "%" + (maxLength + 1) + "d";
	  
	  for (int i = 0; i < n; i++) 
	  {
	    for (int j = 0; j < n; j++) 
	    {
	      System.out.printf(format, matrix[i][j]);
	    }
	    System.out.println();
	  }
  }
  
  public static boolean areEqual(int[][] matrix1, int[][] matrix2)
  {
    if (matrix1.length != matrix2.length || matrix1[0].length != matrix2[0].length)
    {
      return false;
    }

    for (int i = 0; i < matrix1.length; i++)
    {
      for (int j = 0; j < matrix1[0].length; j++)
      {
        if (matrix1[i][j] != matrix2[i][j])
        {
          return false;
        }
      }
    }

    return true;
  }

  public static boolean canMultiply(int[][] matrix1, int[][] matrix2) 
  {
	  int columns = matrix1[0].length;
	  int rows = matrix2.length;
	  return columns == rows;
  }

  public static void main(String[] args) throws InterruptedException 
  {
	  Random ranGen = new Random();
    int n = ranGen.nextInt(15)+5;
	  int[][] matrix1 = generateRandomMatrix(n);
    int[][] matrix2 = generateRandomMatrix(n);
    int[][] CH = new int[n][n];
    int[][] CS = new int[n][n];
    
    Instant starNormal = Instant.now();
    CS = multiplyNormal(matrix1, matrix2);
    Instant endNormal = Instant.now();
    
    Instant startThreaded = Instant.now();
    CH = multiplyThreads(matrix1, matrix2);
    Instant endThreaded = Instant.now();
    
    long normalTime = Duration.between(starNormal, endNormal).toNanosPart();
    long threadedTime = Duration.between(startThreaded, endThreaded).toNanosPart();
    
    System.out.println("Matriz 1:");
    printMatrix(matrix1);
    System.out.println("Matriz 2:");
    printMatrix(matrix2);
    System.out.println("Resultado Secuencial: ");
    printMatrix(CS);
    System.out.println("Resultado Concurrente: ");
    printMatrix(CH);
    
    if(areEqual(CS, CH))
    {
      System.out.println("Las matrices son iguales!");
    }
    else
    {
      System.out.println("Las matrices no son iguales :(");
    }
    	
    System.out.println((double)normalTime/1000 + " microsegundos secuencial");
    System.out.println((double)threadedTime/1000 + " microsegundos con hilos");
    System.out.println(String.format("%.2f ratio hilos/secuencial",(double)threadedTime/normalTime));
  }

}

Writing MatrixMultiplier.java


Ejecutamos el programa.

In [6]:
!java MatrixMultiplier.java

Matriz 1:
 -22  30 -23  -9  -8 -12  17 -17
  12 -23   7  -7 -17  -5 -31 -30
 -11   7  27 -17   5 -22  27  -9
  14  31   1   2 -32  -3   2  -6
  21   4  28 -14   5  -1 -17  26
 -30   1  -3  31  28 -14 -10  23
  29  25   8  28  17   7  13  10
  25  19 -15  30  16  11 -29  14
Matriz 2:
 -32  -4 -13 -23  19  -8   0  -4
  19  -9  25 -30 -23  -7  23  -4
 -15  12   9 -11  13  24  -7   2
  19   5  16   9 -12  28   4 -18
 -19 -30  -5  13  -7  21  -4  26
 -19  -2 -16  -3 -11 -28 -24 -14
  22  18  16  17  -9  15 -13  21
  -1  11  21 -28   2 -27 -14 -28
Resultado Secuencial: 
  2219  -120   832   475 -1298    44  1152   877
 -1293  -160 -1741   381  1325   165   405     1
   683   501   859   435   131  1580   187  1633
   879   623   748 -1450  -231  -645   972  -794
 -1758   -22   120 -1986  1020  -740  -299  -733
  1104  -509  1291   564  -910  1238   200  -378
  -221  -285   989 -1112  -549   655    86  -367
  -809 -1171    69 -1420  -461  -638   515 -1485
Resultado Concurrente: 
  2219  -120 

Viendo los resultados de tiempo obtenidos, vemos que corriendo sobre el sistema de Google Colab la resolucion mediante hilos es **por lo menos** unas **20 veces menos performante** que el metodo secuencial.

Probablemente relacionado a **problemas de overhead** cuando la VM debe cambiar entre hilo e hilo, tiempo que la solucion secuencial gasta haciendo las multiplicaciones necesarias.

Cuando ambas funciones las probamos en una de nuestras PC tambien la version secuencial suele ser mas rapida, pero termina siendo una cuestion aleatoria ya que ambas resoluciones oscilaban en tiempos entre 2000 hasta 5000 microsegundos dependiendo de la corrida.

# Python

El programa realiza un producto matricial de dos formas distintas, primero de forma secuencial y luego concurrente usando hilos.
Obtenemos como conclusión los tiempos de ejecucion transcurridos, calculando el producto matricial de las 2 maneras solicitadas.
Para la realización del ejercicio nos brindamos de las bibliotecas: 
random - para el armado de las matrices aleatorias.
threading - para la utilizacion de hilos.
time- para el calculo de los tiempos de ejecucion.

In [7]:
import random
import threading
import time 

#########################Declaraciones#############################
global A, B
N = 5
A = [[random.randint(-32, 32) for _ in range(N)] for _ in range(N)]
B = [[random.randint(-32, 32) for _ in range(N)] for _ in range(N)]
CS = [[0 for _ in range(N)] for _ in range(N)]
CH = [[0 for _ in range(N)] for _ in range(N)]
####################################################################

#########################Multiplicaciones###########################
def multiplicar_matrices_secuenciales(N):
  for i in range(N):
    for j in range(N):
      for k in range(N):
        CS[i][j] += A[i][k] * B[k][j]

  return CS


def multiplicar_matrices_con_hilos(N):
  def multiplicar_fila_hilo(fila):
    for j in range(N):
      for k in range(N):
        CH[fila][j] += A[fila][k] * B[k][j]

  hilos = []
  for i in range(N):
    hilo = threading.Thread(target=multiplicar_fila_hilo, args=(i,))
    hilos.append(hilo)
    hilo.start()

  for hilo in hilos:
    hilo.join()

  return CH
####################################################################

#########################Evaluadores################################

def comparar_matrices(A, B):
  res = True
  for i in range(N):
    for j in range(N):
      if (A[i][j] != B[i][j]):
        res = False
        break
  return res


############################Ejecuciones#############################

inicio_secuencial = time.time() # registra el tiempo de inicio
resultado_CS = multiplicar_matrices_secuenciales(N)
fin_secuencial = time.time() # registra el tiempo de finalización

inicio_hilo = time.time() # registra el tiempo de inicio
resultado_CH = multiplicar_matrices_con_hilos(N)
fin_hilo = time.time() # registra el tiempo de finalización

tiempo_total_secuencial = fin_secuencial - inicio_secuencial # calcula el tiempo total de ejecución
tiempo_total_hilo = fin_hilo - inicio_hilo # calcula el tiempo total de ejecución
print("\n")
print(f"La función secuencial tardó {tiempo_total_secuencial} segundos en ejecutarse.")
print(f"La función hilo tardó {tiempo_total_hilo} segundos en ejecutarse.")
print("\n")

###################################################################

############################Impresiones#############################
print("Matriz A:")
for fila in A:
  print(fila)
print("\nMatriz B:")
for fila in B:
  print(fila)
print("\nMatriz CH:")
for fila in resultado_CH:
  print(fila)
print("\nMatriz CS:")
for fila in resultado_CS:
  print(fila)

if (comparar_matrices(resultado_CS, resultado_CH)):
  print("\nLas matrices son iguales.")
else:
  print("\nLas matrices son diferentes.")

###################################################################



La función secuencial tardó 0.00015854835510253906 segundos en ejecutarse.
La función hilo tardó 0.004853248596191406 segundos en ejecutarse.


Matriz A:
[-17, 14, 28, -20, 24]
[-25, -5, -30, 28, -29]
[17, 27, -9, 7, -23]
[6, -9, -23, -28, 11]
[28, -19, -20, 25, -32]

Matriz B:
[26, -2, 16, 7, 9]
[-3, 5, 14, 6, 13]
[9, 2, 1, 3, -24]
[19, -17, -9, -20, -6]
[4, 24, 25, 18, -21]

Matriz CH:
[-516, 1076, 732, 881, -1027]
[-489, -1207, -1477, -1377, 871]
[321, -588, 3, -300, 1161]
[-512, 637, 474, 677, 426]
[952, -1384, -863, -1054, 1007]

Matriz CS:
[-516, 1076, 732, 881, -1027]
[-489, -1207, -1477, -1377, 871]
[321, -588, 3, -300, 1161]
[-512, 637, 474, 677, 426]
[952, -1384, -863, -1054, 1007]

Las matrices son iguales.
